In [1]:
import sys

sys.path.append('../..')

import torch
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

from VISTorchUtils import VISDatasetV2
from utils import visCollateV2, batchWaveFromCochleagram
from IPython.display import Audio
from tqdm import tqdm

from VISVMAEModel import VISVMAEModel
from metrics import Evaluator

In [2]:
BATCH_SIZE=4

valDataset = VISDatasetV2('/scratch/vis_data_v2/test')
valDataLoader = DataLoader(valDataset, batch_size=BATCH_SIZE, collate_fn=visCollateV2, shuffle=True, num_workers=6)


/home2/dhruv.kapur/anaconda3/envs/torch/lib/python3.9/site-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


In [3]:
model = VISVMAEModel.load_from_checkpoint('model_weights/vmae-model-epoch=04-val_loss=0.00.ckpt')

/home2/dhruv.kapur/anaconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.1.post0, which is newer than your current Lightning version: v1.9.0
  rank_zero_warn(


In [4]:
preds = []
gts = []

for batch in tqdm(valDataLoader):
    coch, frames, material = batch
    gts += list(coch)
    preds += list(model(frames).cpu().detach().numpy())
    break

  0%|          | 0/1759 [00:08<?, ?it/s]


In [5]:
preds = np.array(batchWaveFromCochleagram(preds))
gts = np.array(batchWaveFromCochleagram(gts))

In [6]:
evaluator = Evaluator(preds, gts)

In [7]:
metrics = evaluator.get_metrics()

0 4


In [8]:
metrics

{'reconstructable': 1.0,
 'reconstruction_loss': 0.06814767686226604,
 'loudness_diff': 0.007044470059391816,
 'spectral_centroid_diff': 2386.240015377467,
 'peak_displacement': 5.864583333333332}